In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np 
import pickle 
import os

os.makedirs('modelo', exist_ok=True)

df = pd.read_csv('data/datos.csv')
df_1 = pd.read_csv('webscrapping/tesla_autoscout24.csv')
df_1

,Unnamed: 0,Modelo,Precio,Color,Pais,Kilometraje,Velocidad
0,0,Model 3,36500.0,Azul,España,19700,377 kW (513 CV)
1,1,Model 3,29700.0,Azul,España,21318,324 kW (441 CV)
2,2,Model 3,25990.0,Azul,España,63000,324 kW (441 CV)
3,3,Model 3,21500.0,Azul,España,129446,239 kW (325 CV)
4,4,Model 3,25850.0,Azul,España,93000,350 kW (476 CV)
...,...,...,...,...,...,...,...
1360,1360,Model Y,38000.0,Gris,Italia,32000,88 kW (120 CV)
1361,1361,Model Y,52000.0,Gris,Italia,18000,380 kW (517 CV)
1362,1362,Model Y,53400.0,Gris,Italia,32000,393 kW (534 CV)
1363,1363,Model Y,44000.0,Gris,Italia,16000,153 kW (208 CV)


In [10]:
df_1 = df_1[['Modelo', 'Precio', 'Color', 'Pais', 'Kilometraje']]
df_1.rename(columns={'Modelo' : 'model', 'Precio' : 'price', 'Color' : 'color', 'Pais': 'country', 'Kilometraje' : 'km'}, inplace=True)
df_1

,model,price,color,country,km
0,Model 3,36500.0,Azul,España,19700
1,Model 3,29700.0,Azul,España,21318
2,Model 3,25990.0,Azul,España,63000
3,Model 3,21500.0,Azul,España,129446
4,Model 3,25850.0,Azul,España,93000
...,...,...,...,...,...
1360,Model Y,38000.0,Gris,Italia,32000
1361,Model Y,52000.0,Gris,Italia,18000
1362,Model Y,53400.0,Gris,Italia,32000
1363,Model Y,44000.0,Gris,Italia,16000


In [11]:
df

,model,price,year,km,color,country
0,Model S,36999.0,2019,116000,Blanco,España
1,Model S,59900.0,2017,56000,Blanco,España
2,Model S,36999.0,2019,116000,Blanco,España
3,Model S,59900.0,2017,56000,Blanco,España
4,Model S,84900.0,2023,13000,Blanco,España
...,...,...,...,...,...,...
1143,Model Y,62620.0,2024,9,Rojo,España
1144,Model Y,62825.0,2024,9,Azul,Portugal
1145,Model Y,64815.0,2024,13,Azul,Portugal
1146,Model Y,66420.0,2024,10,Rojo,España


Preparación de Datos

In [77]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df[['km']])
with open('modelo/scaler_km.pkl', 'wb') as f:
    pickle.dump(scaler, f)
df['km'] = scaler.transform(df[['km']])

In [78]:
le_model = LabelEncoder()
le_model.fit(df[['model']])
with open('modelo/le_model.pkl', 'wb') as f:
    pickle.dump(le_model, f)
df['model'] = le_model.transform(df[['model']])

le_color = LabelEncoder()
le_color.fit(df[['color']])
with open('modelo/le_color.pkl', 'wb') as f:
    pickle.dump(le_color, f)
df['color'] = le_color.transform(df[['color']])

le_year = LabelEncoder()
le_year.fit(df[['year']])
with open('modelo/le_year.pkl', 'wb') as f:
    pickle.dump(le_year, f)
df['year'] = le_year.transform(df[['year']])

le_country = LabelEncoder()
le_country.fit(df[['country']])
with open('modelo/le_country.pkl', 'wb') as f:
    pickle.dump(le_country, f)
df['country'] = le_country.transform(df[['country']])

df

/home/kike/miniconda3/envs/.env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kike/miniconda3/envs/.env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/kike/miniconda3/envs/.env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kike/miniconda3/envs/.env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWa

,model,price,year,km,color,country
0,1,36999.0,5,1.204571,1,0
1,1,59900.0,3,0.164200,1,0
2,1,36999.0,5,1.204571,1,0
3,1,59900.0,3,0.164200,1,0
4,1,84900.0,9,-0.581400,1,0
...,...,...,...,...,...,...
1143,2,62620.0,10,-0.806658,7,0
1144,2,62825.0,10,-0.806658,0,2
1145,2,64815.0,10,-0.806588,0,2
1146,2,66420.0,10,-0.806640,7,0


In [79]:
from sklearn.model_selection import train_test_split

X= df.drop(columns=['price'])
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [81]:
model = RandomForestRegressor(n_estimators=5000, random_state=137)

model.fit(X_train, y_train)
with open('modelo/modelo_entrenado.pkl', 'wb') as f:
    pickle.dump(model, f)
y_pred = model.predict(X_test)

mse = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mse:.2f}")
print(f"R2 Score: {r2:.2f}")

Mean Absolute Error: 3459.47
R2 Score: 0.93
